<a href="https://colab.research.google.com/github/FeraMaks/test/blob/main/NN_ShaftTraectory_mistakes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from numpy.core.memmap import uint8

from google.colab import drive

In [ ]:
#device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
drive.mount('/content/drive/')

Hyper parameters

#Создание датасета

In [ ]:
# Советую воспользоваться циклом for, чтобы не писать множество однотипных строк кода

xy = pd.read_excel(open('/content/drive/MyDrive/work/100.xlsx', 'rb'))
xy = pd.DataFrame(xy)
xynp = pd.DataFrame.to_numpy(xy)
xynp = np.rot90(xynp)
xynp.shape
xynp1 = np.concatenate((xynp[0,:2000], xynp[1,:2000]))
xynp2 = np.concatenate((xynp[0,2000:4000], xynp[1,2000:4000]))
xynp3 = np.concatenate((xynp[0,4000:6000], xynp[1,4000:6000]))
xynp4 = np.concatenate((xynp[0,6000:8000], xynp[1,6000:8000]))
xynp5 = np.concatenate((xynp[0,8000:10000], xynp[1,8000:10000]))
xynp6 = np.concatenate((xynp[0,10000:12000], xynp[1,10000:12000]))
xynp7 = np.concatenate((xynp[0,12000:14000], xynp[1,12000:14000]))
xynp8 = np.concatenate((xynp[0,14000:16000], xynp[1,14000:16000]))
xynp9 = np.concatenate((xynp[0,16000:18000], xynp[1,16000:18000]))
xynp10 = np.concatenate((xynp[0,18000:20000], xynp[1,18000:20000]))

xynp11 = np.concatenate((xynp[2,:2000], xynp[3,:2000]))
xynp12 = np.concatenate((xynp[2,2000:4000], xynp[3,2000:4000]))
xynp13 = np.concatenate((xynp[2,4000:6000], xynp[3,4000:6000]))
xynp14 = np.concatenate((xynp[2,6000:8000], xynp[3,6000:8000]))
xynp15 = np.concatenate((xynp[2,8000:10000], xynp[3,8000:10000]))
xynp16 = np.concatenate((xynp[2,10000:12000], xynp[3,10000:12000]))
xynp17 = np.concatenate((xynp[2,12000:14000], xynp[3,12000:14000]))
xynp18 = np.concatenate((xynp[2,14000:16000], xynp[3,14000:16000]))
xynp19 = np.concatenate((xynp[2,16000:18000], xynp[3,16000:18000]))
xynp20 = np.concatenate((xynp[2,18000:20000], xynp[3,18000:20000]))

xynp21 = np.concatenate((xynp[4,:2000], xynp[5,:2000]))
xynp22 = np.concatenate((xynp[4,2000:4000], xynp[5,2000:4000]))
xynp23 = np.concatenate((xynp[4,4000:6000], xynp[5,4000:6000]))
xynp24 = np.concatenate((xynp[4,6000:8000], xynp[5,6000:8000]))
xynp25 = np.concatenate((xynp[4,8000:10000], xynp[5,8000:10000]))
xynp26 = np.concatenate((xynp[4,10000:12000], xynp[5,10000:12000]))
xynp27 = np.concatenate((xynp[4,12000:14000], xynp[5,12000:14000]))
xynp28 = np.concatenate((xynp[4,14000:16000], xynp[5,14000:16000]))
xynp29 = np.concatenate((xynp[4,16000:18000], xynp[5,16000:18000]))
xynp30 = np.concatenate((xynp[4,18000:20000], xynp[5,18000:20000]))

x = np.vstack((xynp1,xynp2,xynp3,xynp4,xynp5,xynp6,xynp7,xynp11,xynp12,xynp13,xynp14,xynp15,xynp16,xynp17,xynp21,xynp22,xynp23,xynp24,xynp25,xynp26,xynp27,xynp8,xynp9,xynp10,xynp18,xynp19,xynp20,xynp28,xynp29,xynp30))
# Тип данных  в нейросетях - float32. Можно и float64, но тогда модель нужно делать с таким типом данных.
x = x.astype('float32')
x.shape

(30, 4000)

In [ ]:
x[0,:]

array([-182.15427 , -182.81377 , -183.42418 , ...,  -46.86475 ,
        -44.582413,  -42.28893 ], dtype=float32)

In [ ]:
# Индексация классов начиается с нуля. То есть первый класс - 0, второй класс - 1. Тип данных разметки - int64.
# Размерность у разметки должна быть одна
y1 = np.zeros((20), dtype=np.int64)
y2 = np.ones((10), dtype=np.int64)
y = np.concatenate((y1,y2))
y.shape

(30,)

In [ ]:
# Нельзя все объединять в один массив, из-за разных типов данных.

# yx = np.concatenate((y,x), axis=1)
# yx.shape

In [ ]:
# Тут и далее уже очень много ошибок, покажу просто как правильно с комментариями.

class ShaftDataset(Dataset):
    def __init__(self, x, y, mode):
        # Проверка выборки
        if mode == 'train':
            self.x = x[:21]
            self.y = y[:21]
        elif mode == 'val':
            self.x = x[21:]
            self.y = y[21:]
            
        # Разбиение сделано по вашему примеру, но обращаю внимание,
        # что так в тренировочной выборке всего один пример со 2 классом. А в тестовой - только 2 класс/
        # Поэтому нужно как то примеры скомпоновать по-другому

    def __getitem__(self, index):
        return self.x[index], self.y[index]
  
    def __len__(self):
        return len(self.y)

In [ ]:
# Создаем два отдельных датасета и их будем подавать в Loader.
# Изначально вы пытались сделать два среза по одному датасету и подавили в Loader. Так работать не будет

train_dataset = ShaftDataset(x, y, 'train')
test_dataset = ShaftDataset(x, y, 'val')


In [ ]:
train_dataset[0]

(array([-182.15427 , -182.81377 , -183.42418 , ...,  -46.86475 ,
         -44.582413,  -42.28893 ], dtype=float32),
 0)

Hyper parameters

In [ ]:
# input_size = x.shape[0]*x.shape[1] - неправильно. Входной размер равен кол-ву признаков, то есть 4000
input_size = x.shape[1]
hidden_size = 32 # Попробуйте для начала поменьше
num_classes = 2
num_epochs = 10
batch_size = 6 # Bath size не может быть больше размера датасета (у вас 30 примеров)
batch_size = 2
lr = 0.001 # learning rate

In [ ]:
# Loader генерирует тензоры следующего размера -
# (batch_size (размер батча - кол-во примеров в батче), m (кол-во признаков))
# Таким образом, размерностей 2, а не 4

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [ ]:
# Размеры данных в датасетах

features, labels = train_dataset.x, train_dataset.y
print(features.shape, labels.shape)
#samples = torch.from_numpy(samples)
#labels = torch.from_numpy(labels)

features, labels = test_dataset.x, test_dataset.y
print(features.shape, labels.shape)
#samples = torch.from_numpy(samples)
#labels = torch.from_numpy(labels)

(21, 4000) (21,)
(9, 4000) (9,)


#Создание нейронной сети

In [ ]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    return out


In [ ]:
model = NeuralNet(input_size, hidden_size, num_classes)

# Модель также перемещаем в нужное место
model = model.to(device)

Loss and optimizer

In [ ]:
L = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#Training loop

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (features, labels) in enumerate(train_loader):
#     images = images.reshape(-1, 1*4000).to(device) - Видимо из другого кода,
                                                    # здесь не подходит, так как изображений тут нет
    features = features.to(device)
    labels = labels.to(device)

    #forward path
    outputs = model(features) # Подаем признаки на вход нейросети
    loss = L(outputs, labels)

    #backward path
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 5 == 0:
       print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')


epoch 1 / 10, step 5/11, loss = 0.0000
epoch 1 / 10, step 10/11, loss = 21.2478
epoch 2 / 10, step 5/11, loss = 0.0000
epoch 2 / 10, step 10/11, loss = 317.8329
epoch 3 / 10, step 5/11, loss = 203.0966
epoch 3 / 10, step 10/11, loss = 0.0000
epoch 4 / 10, step 5/11, loss = 0.0001
epoch 4 / 10, step 10/11, loss = 0.3226
epoch 5 / 10, step 5/11, loss = 0.3221
epoch 5 / 10, step 10/11, loss = 0.3215
epoch 6 / 10, step 5/11, loss = 0.3207
epoch 6 / 10, step 10/11, loss = 0.0000
epoch 7 / 10, step 5/11, loss = 0.4721
epoch 7 / 10, step 10/11, loss = 0.0000
epoch 8 / 10, step 5/11, loss = 0.0000
epoch 8 / 10, step 10/11, loss = 0.0256
epoch 9 / 10, step 5/11, loss = 0.6954
epoch 9 / 10, step 10/11, loss = 0.3135
epoch 10 / 10, step 5/11, loss = 0.0000
epoch 10 / 10, step 10/11, loss = 0.0000


In [ ]:
outputs, labels

(tensor([[ 0.1117, -0.0369]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([1], device='cuda:0'))

#Test

In [ ]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for features, labels in test_loader:
#     images = images.reshape(-1, x.shape[0]*x.shape[1]).to(device) - опять таки - изображений нет

    features = features.to(device)
    labels = labels.to(device)
    outputs = model(features)
    print(outputs)
    #value, index
    _, predictions = torch.max(outputs, 1)
    n_samples += labels.shape[0]
    n_correct += (predictions == labels).sum().item()
  acc = 100.0 * n_correct / n_samples
  print(f'accuracy = {acc}')

tensor([[ 0.1118, -0.0370],
        [ 0.1118, -0.0370]], device='cuda:0')
tensor([[ 0.1118, -0.0370],
        [ 1.2472,  0.3064]], device='cuda:0')
tensor([[-0.3167,  0.8038],
        [-1.4375,  1.3688]], device='cuda:0')
tensor([[0.6448, 0.1242],
        [2.0859, 1.7976]], device='cuda:0')
tensor([[-4.7792,  2.8541]], device='cuda:0')
accuracy = 33.333333333333336
